# Simulação de um Sistema Solar com um vulcano

## Criação de um dataframe com dos planetas dos SS

In [23]:
# Import modules
import os
import sys
import yaml
import pandas as pd
import numpy as np
import rebound
from oe2pv import orbel_el2xv

## Vetorizando a função de conversão de elementos orbitais

In [24]:
# Transforms orbel_el2xv in vectorized function
orbel_el2xv_vec = np.vectorize(orbel_el2xv)

## Definindo funções

In [25]:
def read_config_file(config_file):
    """
    Read config file and go to home directory
    """
    home_dir = os.path.dirname(os.path.abspath(config_file))
    os.chdir(home_dir)

    # Read yaml file
    with open(config_file, 'r') as stream:
        try:
            config = yaml.load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    
    return config

## Leitura de arquivos de configuração e definição de variáveis

In [26]:
#Read config file
config = read_config_file("config.yaml")

# Read variable in config file
planets_names = config["planets_names"]
vulcans_variants = len(config["vulcans_semi_axis"])
vulcans_clones = config["vulcans_clones"]

# Mass of the Sum [kg]
mass_sun_kg = config["mass_sun_kg"]

# Mass of the Sun, considering G = 1
mass_sun_grav = config["mass_sun_grav"]

# Conic section is ellipse # Constant used in oe2pv function
ialpha = config["ialpha"]

# Gravitational factor of the Sun
gm = config["gm"]

## Criando dataframe inicial dos planetas

In [27]:
# Initial dataframe
for i in planets_names:
    # Create raw dataframe
    exec("{0} = pd.DataFrame(config['{0}'], index = [0])".format(i))

## Incluindo outros dados que se repetirão

In [28]:
# Create gravitational mass
for i in planets_names:
    exec("{0}['mass_grav'] = {0}['mass'] * mass_sun_grav / mass_sun_kg".format(i))

# Create gmpl column
for i in planets_names:
    exec("{0}['gmpl'] = {0}['mass_grav'] + gm".format(i))

# Replicate initial values in each simulate
for i in planets_names:
    exec("{0} = {0}.append([{0}] * (vulcans_variants * vulcans_clones - 1),\
    ignore_index=True)".format(i))

## Criação de dados para os planetas terrestres

In [29]:
# Data for terrestrial planets
terrestrial = planets_names[0:4]
# Create random eccentricity
# Usin numpy.random.ranf. For range = (a,b): (b - a) * random_sample() + a
for i in terrestrial:
    exec("{0}['e'] = 0.01 * np.random.ranf((vulcans_variants * \
    vulcans_clones,))".format(i))

# Create random inclination
# Usin numpy.random.ranf. For range = (a,b): (b - a) * random_sample() + a
for i in terrestrial:
    exec("{0}['inc'] = np.deg2rad(0.01 * np.random.ranf((vulcans_variants *\
    vulcans_clones,)))".format(i))

# Create capom angle
for i in terrestrial:
    exec("{0}['capom'] = np.deg2rad(np.random.randint(0, 361, \
    vulcans_variants * vulcans_clones))".format(i))

# Create omega angle
for i in terrestrial:
    exec("{0}['omega'] = np.deg2rad(np.random.randint(0, 361,\
    vulcans_variants * vulcans_clones))".format(i))

# Create M angle - Mean Anomaly
for i in terrestrial:
    exec("{0}['capm'] = np.deg2rad(np.random.randint(0, 361,\
    vulcans_variants * vulcans_clones))".format(i))

# Create postions and velocities 
for i in terrestrial:
    exec('x, y, z, vx, vy, vz = orbel_el2xv_vec({0}["gmpl"],\
        ialpha,{0}["a"], {0}["e"], {0}["inc"], {0}["capom"],\
        {0}["omega"],{0}["capm"])'.format(i))
    for j in ['x', 'y', 'z', 'vx', 'vy', 'vz']:
        exec("{0}['{1}'] = {1} ".format(i, j))

## Criação dos dados para os planetas gigantes

In [30]:
# Data for giants planets
giants = planets_names[4:8]
sim = rebound.Simulation()
sim.units = ('day', 'AU', 'Msun')
for i in giants:
    sim.add(i) # Read data from NASA
    
for j, p in zip(giants, sim.particles):
    exec("{0}['x'] = {1}".format(j,p.x))
for j, p in zip(giants, sim.particles):
    exec("{0}['y'] = {1}".format(j,p.y))
for j, p in zip(giants, sim.particles):
    exec("{0}['z'] = {1}".format(j,p.z))
for j, p in zip(giants, sim.particles):
    exec("{0}['vx'] = {1}".format(j,p.vx))
for j, p in zip(giants, sim.particles):
    exec("{0}['vy'] = {1}".format(j,p.vy))
for j, p in zip(giants, sim.particles):
    exec("{0}['vz'] = {1}".format(j,p.vz))

# Save planet dataframe
for i in planets_names:
    exec("{0}.to_csv('{0}.csv', index=False)".format(i))

Searching NASA Horizons for 'jupiter'... Found: Jupiter Barycenter (5).
Searching NASA Horizons for 'saturn'... Found: Saturn Barycenter (6).
Searching NASA Horizons for 'uranus'... Found: Uranus Barycenter (7).
Searching NASA Horizons for 'neptune'... Found: Neptune Barycenter (8).


## Verificando arquivos criados
### Mercúrio

In [31]:
read_mercury = pd.read_csv("mercury.csv")
print(read_mercury[:10])

          a          mass     radio     mass_grav      gmpl         e  \
0  0.387099  5.972300e+24  6378.137  8.887539e-10  0.000296  0.002924   
1  0.387099  5.972300e+24  6378.137  8.887539e-10  0.000296  0.003082   
2  0.387099  5.972300e+24  6378.137  8.887539e-10  0.000296  0.006588   
3  0.387099  5.972300e+24  6378.137  8.887539e-10  0.000296  0.007625   
4  0.387099  5.972300e+24  6378.137  8.887539e-10  0.000296  0.004137   
5  0.387099  5.972300e+24  6378.137  8.887539e-10  0.000296  0.003859   
6  0.387099  5.972300e+24  6378.137  8.887539e-10  0.000296  0.007322   
7  0.387099  5.972300e+24  6378.137  8.887539e-10  0.000296  0.007319   
8  0.387099  5.972300e+24  6378.137  8.887539e-10  0.000296  0.005200   
9  0.387099  5.972300e+24  6378.137  8.887539e-10  0.000296  0.006456   

        inc     capom     omega      capm         x         y         z  \
0  0.000112  4.537856  1.221730  2.181662 -0.035641  0.386109 -0.000011   
1  0.000146  0.750492  0.174533  4.660029  0.2

### Júpiter

In [32]:
read_jupiter = pd.read_csv("jupiter.csv")
print(read_jupiter[:10])

          a          mass    period    radio     mass_grav      gmpl  \
0  5.203363  1.898190e+27  4332.589  71492.0  2.824747e-07  0.000296   
1  5.203363  1.898190e+27  4332.589  71492.0  2.824747e-07  0.000296   
2  5.203363  1.898190e+27  4332.589  71492.0  2.824747e-07  0.000296   
3  5.203363  1.898190e+27  4332.589  71492.0  2.824747e-07  0.000296   
4  5.203363  1.898190e+27  4332.589  71492.0  2.824747e-07  0.000296   
5  5.203363  1.898190e+27  4332.589  71492.0  2.824747e-07  0.000296   
6  5.203363  1.898190e+27  4332.589  71492.0  2.824747e-07  0.000296   
7  5.203363  1.898190e+27  4332.589  71492.0  2.824747e-07  0.000296   
8  5.203363  1.898190e+27  4332.589  71492.0  2.824747e-07  0.000296   
9  5.203363  1.898190e+27  4332.589  71492.0  2.824747e-07  0.000296   

          x         y         z        vx        vy        vz  
0 -5.014065 -2.128981  0.120977  0.002861 -0.006588 -0.000037  
1 -5.014065 -2.128981  0.120977  0.002861 -0.006588 -0.000037  
2 -5.014065 -2.

## Comparando com os dados diretos da Horizons

<https://ssd.jpl.nasa.gov/horizons.cgi#top>

In [33]:
with open("Jupiter_verification_Horizons.txt", "r") as f:
    print(f.read())

*******************************************************************************
Ephemeris / WWW_USER Mon Jun 12 14:00:00 2017 Pasadena, USA      / Horizons    
*******************************************************************************
Target body name: Jupiter (599)                   {source: jup340_merged}
Center body name: Solar System Barycenter (0)     {source: DE431mx}
Center-site name: BODY CENTER
*******************************************************************************
Start time      : A.D. 2017-Jun-12 00:00:00.0000 TDB
Stop  time      : A.D. 2017-Jun-13 00:00:00.0000 TDB
Step-size       : 1440 minutes
*******************************************************************************
Center geodetic : 0.00000000,0.00000000,0.0000000 {E-lon(deg),Lat(deg),Alt(km)}
Center cylindric: 0.00000000,0.00000000,0.0000000 {E-lon(deg),Dxy(km),Dz(km)}
Center radii    : (undefined)                                                  
Output units    : AU-D                               